In [1]:
# Solving the stochastic Optimal Growth model by Policy Function Interation 
# adapted from Fabrice Collard's Matlab code, http://fabcol.free.fr/
# tested in Julia 1.5.0
# this code is part of chapter 4, "Dynamic Programming" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com

#packages
using Plots;
using LinearAlgebra;

sigma   = 1.50;                     # utility parameter
delta   = 0.10;                     # depreciation rate
beta    = 0.95;                     # discount factor
alpha   = 0.30;                     # capital elasticity of output
p       = 0.9;
PI      = [p 1-p;1-p p];
se      = 0.2;
ab      = 0;
am      = exp(ab-se);
as      = exp(ab+se);
A       = [am as];
nba     = 2;


In [4]:
nbk     = 1000;                     # number of data points in the grid
crit    = 1;                        # convergence criterion
iter    = 1;                        # iteration
tol     = 1e-6;                     # convergence parameter

ks      = ((1-beta*(1-delta))/(alpha*beta))^(1/(alpha-1));

kmin    = 0.2;                      # lower bound on the grid
kmax    = 6;                        # upper bound on the grid
devk    = (kmax-kmin)/(nbk-1);      # implied increment
kgrid   = collect(LinRange(kmin,kmax,nbk)); # builds the grid
v       = zeros(nbk,nba);           #  value function
u       = zeros(nbk,nba);           # utility function
u_temp  = zeros(nbk,nba);
c_temp  = zeros(nbk,1);
c       = zeros(nbk,1);
Tv      = zeros(nbk,nba);
Ev      = zeros(nbk,nba);

kp0     = repeat(kgrid,1,nba);      # initial guess on k(t+1)
kp      = zeros(nbk,nba)
dr      = zeros(nbk,nba);           # decision rule (will contain indices)

In [5]:
for i=1:nbk;    
   for j=1:nba;       
         c_temp   = A[j]*kgrid[i]^alpha+(1-delta)*kgrid[i].-kgrid;
         neg      = findall(x -> x <=0.0,c_temp);     
         c_temp[neg].= NaN;
         u_temp[:,j]   = (c_temp.^(1-sigma).-1)/(1-sigma);
    
         u_temp[neg,j].= -1e30;    
         Ev[:,j]      = v*PI[j,:];
         
    end
        u_obj = u_temp+beta* Ev;
   
        (tv1,dr1) = findmax(u_obj[:,1], dims=1)
        (tv2,dr2) = findmax(u_obj[:,2], dims=1)

        dr[i,1] = dr1[1]; dr[i,2] = dr2[1];
        Tv[i,1] = tv1[1]; Tv[i,2] = tv2[1]; 
end;

In [6]:
#Main Loop
while crit>tol
for i=1:nbk;    
   for j=1:nba;       
         c_temp   = A[j]*kgrid[i]^alpha+(1-delta)*kgrid[i].-kgrid;
         neg      = findall(x -> x <=0.0,c_temp);     
         c_temp[neg].= NaN;
         u_temp[:,j]   = (c_temp.^(1-sigma).-1)/(1-sigma);
    
         u_temp[neg,j].= -1e30;    
         Ev[:,j]      = v*PI[j,:];
         
    end
        u_obj = u_temp+beta* Ev
   
        (tv1,dr1) = findmax(u_obj[:,1], dims=1)
        (tv2,dr2) = findmax(u_obj[:,2], dims=1)

        dr[i,1] = dr1[1]; dr[i,2] = dr2[1];
        Tv[i,1] = tv1[1]; Tv[i,2] = tv2[1]; 
end;


#decision rules
for i=1:nbk 
   for j=1:nba 
   
   index=trunc(Int, dr[i,j]);
   kp[i,j]= kgrid[index];
        
   end;       
end;
    
Q  = zeros(nbk*nba,nbk*nba);

for j=1:nba;

    c = A[j]*kgrid.^alpha+(1-delta)*kgrid-kp[:,j];
    
#update the value 
    u[:,j]= (c.^(1-sigma).-1)/(1-sigma);
    
    Q0  = zeros(nbk,nbk);
            
    for i=1:nbk;
        index=trunc(Int, dr[i,j]);

        Q0[i,index] = 1;
    end;

Q[(j-1)*nbk+1:j*nbk,:]= kron(PI[j,:]',Q0);
end;

AA   = (Matrix{Float64}(I, nbk*nba, nbk*nba)-beta*Q);
BB   = vec(u);
Tvv  = \(AA, BB);    
crit = maximum(abs.(kp-kp0));
vv   = reshape(Tvv,nbk,nba);
v    = copy(vv);
kp0  = copy(kp);
iter = iter+1;
    
end;    

In [7]:
plotly() # Choose the Plotly.jl backend for web interactivity
#plot(k',c',linewidth=1,label="Consumption",title="Consumption vs capital stock")
plot(kp,v,linewidth=1,label="Value Function vs Capital Stock", title="Value Function")

┌ Warning: Failed to load integration with PlotlyBase & PlotlyKaleido.
│   exception = (ArgumentError("Package PlotlyKaleido not found in current path.\n- Run `import Pkg; Pkg.add(\"PlotlyKaleido\")` to install the PlotlyKaleido package."), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007ffc91685dd4, Ptr{Nothing} @0x00007ffc90f89364, Ptr{Nothing} @0x00007ffca96096fe, Ptr{Nothing} @0x00007ffc9138be56, Ptr{Nothing} @0x00007ffc90fc4314, Ptr{Nothing} @0x00007ffca963a092, Ptr{Nothing} @0x00007ffca963bf66, Ptr{Nothing} @0x00007ffca961db6a, Ptr{Nothing} @0x00007ffca961e39e, Base.InterpreterIP in top-level CodeInfo for Main at statement 1, Ptr{Nothing} @0x00007ffca963b2b8, Ptr{Nothing} @0x00007ffca963ca36, Ptr{Nothing} @0x0000029684aa393e, Ptr{Nothing} @0x0000029684aa4138, Ptr{Nothing} @0x0000029684aa4286, Ptr{Nothing} @0x0000029684aa42b4, Ptr{Nothing} @0x00007ffca961ccfa, Ptr{Nothing} @0x00007ffca961c7f4, Ptr{Nothing} @0x00007ffca961d440, Ptr{Nothing} @0x00007ffca961e39e, Base.In